In [ ]:
# -*- coding: utf-8 -*-
# Extração Beyoung (collections/skincare) — célula única para Jupyter
# Requisitos: requests, selectolax

import re
import csv
import time, sys, os, unicodedata
from typing import List, Dict, Optional
from urllib.parse import urljoin

import requests
from selectolax.parser import HTMLParser
sys.path.append(os.path.abspath(".."))

from skin import (
    SKIN_TYPE_CANONICAL_ORDER,
    SKIN_TYPE_SYNONYMS_PT,
)

from exclude import (
    EXCLUDE_KEYWORDS,
)

from ingredient import (
    INGREDIENTES_VALIDOS,
)

from benefits import (
    BENEFIT_SYNONYMS_PT,
    BENEFIT_CANONICAL_ORDER,
)


# ==== Configs básicas ====
BASE = "https://www.beyoung.com.br"
COLLECTION_URL = "https://www.beyoung.com.br/collections/skincare"
SITE_LABEL = "beyoung"
OUT_CSV = "beyoung_skincare.csv"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept-Language": "pt-BR,pt;q=0.9,en;q=0.8"
}

# ==== Utils ====
def get_html(url: str) -> Optional[HTMLParser]:
    try:
        resp = requests.get(url, headers=HEADERS, timeout=30)
        if resp.ok:
            return HTMLParser(resp.text)
    except Exception:
        pass
    return None

def text(node) -> str:
    return (node.text().strip() if node else "").strip()

def norm_spaces(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def strip_accents(s: str) -> str:
    if not s: return ""
    return "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")

def norm_for_match(s: str) -> str:
    s = norm_spaces(s).lower()
    s = strip_accents(s)
    return s

def price_to_float(s: str) -> Optional[float]:
    if not s:
        return None
    m = re.search(r"(\d{1,3}(?:\.\d{3})*|\d+)(?:,(\d{2}))?", s.replace("\xa0"," ").replace("\n"," "))
    if not m:
        return None
    inteiro = m.group(1).replace(".", "")
    centavos = m.group(2) or "00"
    try:
        return float(f"{inteiro}.{centavos}")
    except Exception:
        return None

def has_excluded_keyword(name: str) -> bool:
    n = norm_for_match(name)
    for kw in EXCLUDE_KEYWORDS:
        if kw and norm_for_match(kw) in n:
            return True
    return False

def guess_quantity(name: str, fallback: str = "") -> str:
    patterns = [
        r"(\d+)\s*(ml|g|kg|l|L)\b",
        r"(\d+,\d+)\s*(ml|g|kg|l|L)\b",
        r"\b(\d{2,4})\s*(ml)\b",
    ]
    for pat in patterns:
        m = re.search(pat, name, flags=re.IGNORECASE)
        if m:
            return "".join(m.groups()).replace(" ", "")
    return fallback.strip()

# ==== Coleta de URLs da coleção ====
def extract_product_urls_from_collection(doc: HTMLParser) -> List[str]:
    urls = set()

    for a in doc.css("a.full-unstyled-link"):
        href = a.attributes.get("href", "")
        if href and "/products/" in href:
            urls.add(urljoin(BASE, href))

    for a in doc.css("a.card__heading, a.product-grid-item, a.product-item"):
        href = a.attributes.get("href", "")
        if href and "/products/" in href:
            urls.add(urljoin(BASE, href))

    for a in doc.css("a"):
        href = a.attributes.get("href", "")
        if href and "/products/" in href:
            urls.add(urljoin(BASE, href))

    return sorted(urls)

# ==== Classificação de BENEFÍCIOS (models) ====
def collect_benefits_text(doc: HTMLParser) -> str:
    """
    Varre trechos típicos de descrição para montar um texto base
    onde buscaremos sinônimos de benefícios.
    """
    parts = []

    # Blocos comuns de descrição em Shopify
    for sel in [
        ".product__description", ".product__description.rte", ".rte",
        ".product__accordion", ".accordion__content", ".product__text",
        "section, article"
    ]:
        for n in doc.css(sel):
            t = norm_spaces(n.text())
            if t and len(t) > 40:
                parts.append(t)

    joined = " ".join(parts)
    if not joined:
        joined = norm_spaces(doc.body.text() if doc.body else "")
    return joined

def classify_benefits(doc: HTMLParser) -> str:
    """
    Retorna rótulos canônicos (separados por vírgula) em ordem de BENEFIT_CANONICAL_ORDER.
    Se nada casado, retorna string vazia.
    """
    txt = norm_for_match(collect_benefits_text(doc))
    found = set()

    for canonical, synonyms in BENEFIT_SYNONYMS_PT.items():
        for syn in synonyms:
            if syn and norm_for_match(syn) in txt:
                found.add(canonical)
                break

    if not found:
        return ""
    ordered = [b for b in BENEFIT_CANONICAL_ORDER if b in found]
    return ", ".join(ordered)

# ==== Filtragem de INGREDIENTES (models) ====
def collect_ingredients_text(doc: HTMLParser) -> str:
    """
    Junta trechos que costumam listar composição/ativos.
    """
    parts = []
    # Títulos/âncoras comuns
    anchors = ("COMPOSIÇÃO", "COMPOSICAO", "INGREDIENTES", "ATIVOS", "PRINCIPAIS ATIVOS")
    for node in doc.css("strong, b, h1, h2, h3"):
        title = norm_spaces(node.text()).upper()
        if any(a in title for a in anchors):
            # pega alguns irmãos seguintes
            hops, cur = 0, node
            while cur and hops < 12:
                cur = cur.next
                if not cur: break
                try:
                    # cur pode ser Node ou str; só pega Node com .text()
                    t = getattr(cur, "text", None)
                    if t:
                        val = norm_spaces(cur.text())
                        if val:
                            parts.append(val)
                except Exception:
                    pass
                hops += 1

    if not parts:
        # fallback: corpo todo
        parts = [norm_spaces(doc.body.text() if doc.body else "")]
    return " ".join(parts)

def filter_ingredients(doc: HTMLParser) -> str:
    """
    Retorna apenas ingredientes presentes em INGREDIENTES_VALIDOS (ordem alfabética).
    """
    txt = norm_for_match(collect_ingredients_text(doc))
    hits = set()
    for ing in INGREDIENTES_VALIDOS:
        if norm_for_match(ing) in txt:
            hits.add(ing)
    if not hits:
        return ""
    # ordena alfabeticamente desconsiderando acentos/caixa
    return ", ".join(sorted(hits, key=lambda s: strip_accents(s).lower()))

# ==== Extração de um produto ====
def extract_product_data(url: str) -> Optional[Dict]:
    doc = get_html(url)
    if not doc:
        return None

    # Nome
    name_node = doc.css_first("h1.product__title") or doc.css_first("h1.product__title.hd3")
    name = text(name_node)

    # Subtítulo
    subtitle_node = doc.css_first("p.product__text.inline-richtext") or doc.css_first(".product__subtitle, .product__text")
    subtitle = text(subtitle_node)

    # Preço
    price_selectors = [
        "span.f-price-item.f-price-item--sale",
        "span.price-item.price-item--sale",
        "span.price-item.price-item--regular",
        "span.money",
        "[data-product-price] .price-item--sale",
        ".price__container .price-item--sale",
        ".price__regular .price-item--regular",
    ]
    raw_price = ""
    for sel in price_selectors:
        node = doc.css_first(sel)
        if node and node.text().strip():
            raw_price = node.text().strip()
            break
    price = price_to_float(raw_price)

    # Quantidade
    qty_node = doc.css_first('[data-selected-swatch-value="Tamanho"]') or doc.css_first("label[for*='template'][for*='main'][for*='-0']")
    quantity = text(qty_node) or guess_quantity(name, "")

    # Filtros
    if not name or has_excluded_keyword(name):
        return None

    # >>> NOVO: Benefícios e Ingredientes padronizados <<<
    beneficios = classify_benefits(doc)       # só rótulos canônicos do models
    ingredientes = filter_ingredients(doc)    # só itens whitelist do models

    return {
        "site": SITE_LABEL,
        "nome": name,
        "subtitulo": subtitle,
        "preco": f"{price:.2f}" if price is not None else "",
        "quantidade": quantity,
        "beneficios": beneficios,     # <- apenas os canônicos reconhecidos
        "ingredientes": ingredientes, # <- apenas os whitelist do models
        "url": url
    }

# ==== Paginação da coleção ====
def paginate_collection(base_url: str, sleep_s: float = 0.8, max_pages: int = 50) -> List[str]:
    all_urls = set()
    page = 1
    while page <= max_pages:
        url = f"{base_url}?page={page}"
        doc = get_html(url)
        if not doc:
            break
        urls = extract_product_urls_from_collection(doc)
        if not urls:
            if page == 1 and base_url != url:
                doc0 = get_html(base_url)
                if doc0:
                    urls0 = extract_product_urls_from_collection(doc0)
                    for u in urls0:
                        all_urls.add(u)
            break
        for u in urls:
            all_urls.add(u)
        page += 1
        time.sleep(sleep_s)
    return sorted(all_urls)

# ==== Escrita CSV ====
def write_csv(rows: List[Dict], path: str):
    fieldnames = ["site", "nome", "subtitulo", "preco", "quantidade", "beneficios", "ingredientes", "url"]
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        for r in rows:
            w.writerow({k: r.get(k, "") for k in fieldnames})

# ==== Execução ====
all_product_urls = paginate_collection(COLLECTION_URL)
print(f"URLs de produtos encontrados na coleção: {len(all_product_urls)}")

rows = []
for i, purl in enumerate(all_product_urls, 1):
    data = extract_product_data(purl)
    if data:
        rows.append(data)
    time.sleep(0.6 if i % 3 else 1.0)

write_csv(rows, OUT_CSV)
print(f"Registros válidos coletados: {len(rows)}")
print(f"CSV salvo em: {OUT_CSV}")

# Visualização de amostra
for r in rows[:5]:
    print(r)


URLs de produtos encontrados na coleção: 25
Registros válidos coletados: 23
CSV salvo em: beyoung_skincare.csv
{'site': 'beyoung', 'nome': 'Sérum facial com Retinol + Niacinamida', 'subtitulo': 'Aging Care', 'preco': '129.90', 'quantidade': '30ml', 'beneficios': 'hidratação, limpeza, antissinais, uniformiza o tom, antioxidante, esfoliação, acalma, luminosidade, fortalece a barreira, proteção solar, fortalece os fios, compatível com maquiagem, resultados rápidos', 'ingredientes': 'ácido cítrico, ácido glicólico, ácido hialurônico, ácido linoleico, ácido salicílico, ácido tranexâmico, alpha arbutin, cafeína, esqualano, fenoxietanol, hipoalergênico, lecitina, lha, niacinamida, pantenol, propanodiol, retinol, vitamina c, vitamina e', 'url': 'https://www.beyoung.com.br/products/aging-care'}
{'site': 'beyoung', 'nome': 'Água Micelar Hidratante', 'subtitulo': 'Micellar Water', 'preco': '44.86', 'quantidade': '200ml', 'beneficios': 'hidratação, limpeza, controle da oleosidade, antissinais, ant